In [12]:
import pandas as pd
import numpy as np
import re
import os

In [13]:
filename = 'twitter-larger.tsv'

In [14]:
def load_data():
    columns = ['timestamp', 'user', 'tweet']
    df = pd.read_table(filename, header=None)
    df.columns = columns
    return df

In [26]:
def create_user_hashtag_df(df):
    columns = ['user', 'hashtag', 'timestamp']
    users_hashtag = []
    
    def extract_hash_tags(tweet):
        return set([re.sub(r"(\W+)$", "", j) for j in set([i for i in tweet.split() if i.startswith("#")])])

    for row in df.itertuples():
        tweet = row.tweet
        #print(tweet)
        hashtags = extract_hash_tags(tweet)
        if(len(hashtags) > 0):
            for hashtag in hashtags:
                if(len(hashtag) == 0):
                    continue
                hashtag = hashtag[1:].lower()
                comb = (row.user, hashtag, row.timestamp)
                users_hashtag.append(comb)
    
    users_hashtag_df = pd.DataFrame(users_hashtag, columns=columns)
    
    return users_hashtag_df

In [21]:
def create_raw_source_target_df(df):
    lst = df['hashtag'].unique()
    users_hashtag = []

    for i in lst:
        hashtag_df = df[df['hashtag'] == i].reset_index()
        copy_hashtags = hashtag_df.copy()
        copy_hashtags = copy_hashtags.iloc[1:,:]
        for index_1, row_1 in hashtag_df.iterrows():
            length = len(copy_hashtags)
            for i in range(length):
                if(len(copy_hashtags) == 0):
                    continue
                max_ts = max(row_1['timestamp'], copy_hashtags.iloc[0,3])
                users = sorted((row_1['user'], copy_hashtags.iloc[0,1]))
                comb = (users[0], users[1], row_1['hashtag'], max_ts)
                users_hashtag.append(comb)
                copy_hashtags = copy_hashtags.iloc[1:,:]
    raw_source_target_df = pd.DataFrame(users_hashtag, columns=['source', 'target', 'hashtag', 'timestamp'])
    
    return raw_source_target_df

In [22]:
def create_raw_source_target_df(df):
    lst = df['hashtag'].unique()
    users_hashtag = []

    for i in lst:
        hashtag_df = df[df['hashtag'] == i].reset_index()
        copy_hashtags = hashtag_df.copy()
        copy_hashtags = copy_hashtags.iloc[1:,:]
        for index_1, row_1 in hashtag_df.iterrows():
            length = len(copy_hashtags)
            for i in range(length):
                if(len(copy_hashtags) == 0):
                    continue
                max_ts = max(row_1['timestamp'], copy_hashtags.iloc[0,3])
                users = sorted((row_1['user'], copy_hashtags.iloc[0,1]))
                comb = (users[0], users[1], row_1['hashtag'], max_ts)
                users_hashtag.append(comb)
                copy_hashtags = copy_hashtags.iloc[1:,:]
    raw_source_target_df = pd.DataFrame(users_hashtag, columns=['source', 'target', 'hashtag', 'timestamp'])
    
    return raw_source_target_df

In [23]:
def create_hashtag_df(df, hashtag):
    iphone = df[df['hashtag'] == hashtag].copy()
    iphone_raw = create_raw_source_target_df(iphone)
    iphone_raw['combined'] = iphone_raw['source'] + ' + ' + iphone_raw['target']
    iphone_raw['weight'] = iphone_raw.groupby('combined')['combined'].transform('count')
    iphone_raw['combined'] = iphone_raw['source'] + ' + ' + iphone_raw['target']
    iphone_raw['weight'] = iphone_raw.groupby('combined')['combined'].transform('count')
    iphone_raw = iphone_raw.sort_values(by=['timestamp'])
    iphone_raw = iphone_raw.drop_duplicates(subset=['combined'])
    final = iphone_raw.drop(columns = ['combined', 'hashtag'])
    final = final[['source', 'target', 'weight', 'timestamp']]
    final['timestamp'] = pd.to_datetime(final['timestamp'])
    final['timestamp'] = final['timestamp'].astype('int64')//1e9
    final['timestamp'] = final['timestamp'].astype(int)
    final['timestamp'] = final['timestamp'] - final['timestamp'].min()
    final = final.sort_values(by=['source'])
    final['source'] = final['source'].apply(lambda x: hash(tuple(x)))
    final['target'] = final['target'].apply(lambda x: hash(tuple(x)))
    final['source'] = final['source'].abs()
    final['source'] = final['source'] % 2**12
    final['target'] = final['target'].abs()
    final['target'] = final['target'] % 2**12
    final.to_csv('Clean/Twitter_' + hashtag + '.txt', sep=' ', index=False, header=False)

In [24]:
df = load_data()
df = create_user_hashtag_df(df)
#df = pd.read_csv('raw_hashtags.csv')
create_hashtag_df(df, 'wpwebhost')

Каждый год в августе я меняю сотовый, похоже сменю Nokia E71 на новый Apple Iphone 3GS, очень интересный аппарат получился...
O que vocês acham: iPhone ou HTC TYTN II? #pesquisa
How to cheat on any test easy http://bit.ly/10ZDwR swine flu h1n1 iphone air france flight father's day dallas
@beatrizchavez A dork? NO way... you are. I just have no shame... I #squarespace want #squarespace that #squarespace iPhone #squarespace.
#iPhone: iPhone 3G S: de jacht op simlockvrije toestellen is geopend http://tinyurl.com/mvky8g
Swine Flu Letterman: An Italian iPhone Father's day. H1N1...Dallas.
More iPhoneLaunchParty photos: @blakespot + @arnoldkim - http://yfrog.com/29borj Crowd - http://yfrog.com/5dguoj
Swine Flu Letterman: An Italian iPhone Father's day. H1N1...Dallas. http://bit.ly/JSKx1
@kerotab oh if you iphone people can dish it you damn well better take it! Haha &lt;3
so many things to look forward to this month... new iphone... entourage season 5 on dvd and warped tour :)
@iphonefan 今のiPh

@lostonroute66 oooooh so jealous. i'd love to replace my craptastic BB Storm with the iPhone 3GS. too bad VZW doesn't support it. :(
iPhone Trailler! http://tinyurl.com/rxwtyj/
http://twitpic.com/74oby (via @MGiraudOfficial) Awwww! @dannygokey and I are iPhone case twins. *tear*
@espyder nel deveras que no caché nada! la verdad ke si le llevo ganas al iphone 3gs!!
RT @st1300a4: thinks that ATT sucks completely for charging loyal customers more for their new iPhone 3G S model.
#squarespace I am still keeping my fingers crossed on the prospect of winning the new iPhone 3GS? http://www.charlesaki.com
#squarespace regala un certificado de $199 en AppleStore. Alcanza para un iPhone.
Petition: AT&T to offer reasonable iPhone 3GS upgrade prices http://twitition.com/f96aq @ryanbarr
Just found out about this daily contest for a free iPhone. Pick me! #squarespace
RT @mashable Got My Mac On With iPhone 3G S [Music Video] - http://bit.ly/QKBC3
RT AllThingsiPhone #iPhone Microsoft and Bing Cashback

RT 21 iPhone Apps For Business| via @sitepointdotcom | http://bit.ly/uJ6mO
think I need to find out more about this iphone blurb going the rounds .. however I didn't how to turn the bloody thing off let alone update
Getting closer to today's 5:00 iPhone winner from #squarespace. I'm getting excited for them to announce my name.
Will AT&T Price-Gouge iPhone Users With MMS & Tethering Surcharges?: InfoWorld's Bill Snyder questions whether A.. http://tinyurl.com/ksh2t2
Crave: iPhone developer: 3G S graphics will be SOOO much better than PSP's http://bit.ly/19qR3f
@marc_mccombs Itunes and the app store is what makes the iphone.
iPhone App Tom Tom http://bit.ly/l69ZW
iPhone: get your updates here sign up and get some great stuff along the wayhttp://bit.ly/7V8y7
#squarespace !!!!!!!!!!! PLEASSSSSEEE I NEED THIS IPHONE!!!!!!
#gmail Google Improves Gmail For Android, iPhone - InformationWeek: IntoMobileGoogle Improves Gmail For An.. http://tinyurl.com/llcoum
#squarespace. It's almost #squaresp

@squarespace, r they still doing a contest today? Win a free iphone at #squarespace
My profile picture is a free iphone i recieved using the sites i promote. Claim yours at ---> http://is.gd/YvHA
@hanabanane But the MacBook matches your pretty white iPhone..! Does the shiny screen or backlit keys bother you?
Steve Sussex has added "Apple iPhone 3G with 8GB Memory - Black" to trade on Commuto http://www.commuto.com/product/i3717/
Google Improves Gmail For Android, iPhone - InformationWeek #test http://cli.gs/s4VMWa
JonJ: #wwdc Hands on - although perhaps hand off more appropriate - Mass Effect Galaxy iPhone; http://tinyurl.com/npqzu8
iPhone Peggle on sale for only a dollar? Now they've gone and done it.
@quickpwn i jist entered to win a free iphone 3g s from quickpwn, you can too! Go to http://bit.ly/iphone-3g-s to enter #quickpwn #iphone
twitition to at&t to offer reasonable prices for iphone 3Gs has doubled since yesterday. http://twitition.com/f96aq
maybe spell it right first *iphone

Back row center of iPhone memory management. #wwdc http://bit.ly/5np1J
RT @Lextar: Rezepte-App für iPhone nur für kurze Zeit im Preis gesenkt (€1,59)! Weitersagen! #iphone #rezepte #kochen
Petition: AT&T to offer reasonable iPhone 3GS upgrade prices http://twitition.com/f96aq @ryanbarr
iphone OS 3.0 beta testers get Find My iphone http://bit.ly/7piTM
How mad are iPhone customers at AT&T? The ways are counted! http://tinyurl.com/m4kroy (via @JasonOGrady) #iphone
Petition: AT&T to offer reasonable iPhone 3GS upgrade prices http://twitition.com/f96aq @ryanbarr
Chastity Bono #MP2 Swine Flu iPhone Lakers Krod Mandoon H1N1 Final Destination PS3 Kobe http://bit.ly/wWm1c http://bit.ly/xTCgj
Are you searching for iPhone, go here http://www.nway.org!!
iphone app: Toki Tori - Chillingo Ltd: Toki Tori Chillingo Ltd Genre: Games Price: $4.99 Release Date.. http://tr.im/od2Z
Is it me or does 3.0 seem to extend the battery life on the iPhone?
Twittelator Pro is een iPhone Twitter client die steeds be

...

說 晚上總是特別餓... （昨天爸爸丟出：只要瘦到50公斤,就給正牌iPhone! 不是山寨的唷！） http://plurk.com/p/10nw7z
@MladenPrajdic iPhone sdk development. Reading it's description on obj c reminds me of why I quit both c++ programming and shrooms
RT @applereport[via @appleinsider] AT&T already sold out of launch day iPhone 3G S pre-orders: Signaling a l.. http://tr.im/oo69
皆iPhone3GS買うのん？ http://bctiny.com/ppz5x
Twitting from iphone!
Mmm frosted flakes. Mmm free iphone #squarespace
RT @JohnAndrew: Looks like Tweetie on iPhone has some minor bugs since the Twitpocalypse. <--heads up peeps
You want to Go to College For Free! Check it out here! http://sldollar.notlong.com apple iphone Brittneyswine flu
Is full-flash encrytpion in 3.0? RT @mubix RT @devilok: "YouTube - iPhone Forensics-Live Recovery over USB" http://bit.ly/nokoc
Ok Ok, here is the tiny link for iPhone guided tour - http://tinyurl.com/54ap5h
@ewebbs oh! Good to know about the iPhone look w/my blog! Thanks and cheers to crazy driving! :) have a great weekend!
Pet

RT @BrianReich: I should be napping, but I just learned the NPR app for the iPhone has transcripts of everything. So much to read! Awesome.
RT @appleinsider [News] AT&T already sold out of launch day iPhone 3G S pre-orders http://tinyurl.com/ngv7ra {hope Rogers can do better!}
Whoever Steals Your iPhone Can Just Turn Off Find My iPhone Location Tracking [IPhone] http://bit.ly/14t8mW
iphone blackout http://twitter.5a5.us/
RT @EpicKid: iPhone 3Gs pre-orders just sold out. There's still a week to go. Wow. (Just read: http://sn.im/k1x5i)
From MacRumors: AT&T Sells Out of Initial Batch of iPhone 3Gs Pre-Orders: BoyGeniusReport claims.. http://tinyurl.com/mh7a5l
Twitter Gone Wild. Find out here! http://tweet4.notlong.com apple iphone Brittney swine flu
iPhone 3G VS BlackBerry 8830: Transition, Keyboard, Activation, and more. http://bit.ly/16UtUX
MacRumors.com: AT&T Sells Out of Initial Batch of iPhone 3Gs Pre-Orders http://tinyurl.com/q6nohw Full http://tinyurl.com/mh7a5l
MacWorld News Feed:

RT @engadget: AT&T's iPhone 3G S pre-orders already sold out http://tinyurl.com/lzyppo
RT @squarespace: Happy Saturday Twitterverse...#Squarespace iPhone gift card giveaway coming in 2.5 hours!
@mashable Twittelator pro for iPhone unaffected by "Twitpocalypse". What was the fuss about?
T-Mobile Accidentally Posts Secret iPhone 3G S Specs http://globalitnews.blogspot.com/ #tmoblie #iphone #holland #globalITNews
RT @squarespace: Happy Saturday Twitterverse...#Squarespace iPhone gift card giveaway coming in 2.5 hours! - Not long!
RT @chrispirillo -- iPhone 3G vs the Pre: http://bit.ly/Vm4qr [If you have a Pre, what do you think?]
I need an Iphone just because. #squarespace
Use 37 Signals Backpack application? Have an iPhone? http://violetpixel.com/ibp/
Interesting - Twitter "twitpocalypse" affects Mac, iPhone apps (Macworld.com) : Macworld.com - Th.. http://ping.fm/xItV1
Your next iPhone: iPhone 3.0 update or iPhone 3G S? (InfoWorld) http://u.mavrev.com/amqy
RT @engadget: AT&T's iPhone 3G

Week in Apple: WWDC mania! iPhone 3G S, Snow Leopard, Safari 4, more http://bit.ly/Lydho
20 Doom Resurrection برای Iphone: وین بتا http://tinyurl.com/mrtslj
Squarespace is giving away an iPhone a day for 30 days 2 a random person who hashtags #squarespace : http://www.squarespace.com/iphone
RT @Twitterrific: Twitterrific 2.0.1 for the iPhone is indeed experiencing problems with the Twitpocalypse. Investigating now...
RT razorianfly Trying to convince @Squarespace to give me an iPhone. [ ] < Is it enough? Is it? IS IT? It is, isn't it? Yay! / #squarespace
Petition: Rogers Wireless to to offer early iPhone 3GS upgrades for existing subscribers http://twitition.com/wqmww @iphoneincanada
@quixoticblazes Sorry, you tweet too quickly. I was referring to the iPhone tweet! ;-)
@theweene best get iphone innit ;)
RT @squarespace: Happy Saturday Twitterverse...#Squarespace iPhone gift card giveaway coming in 2.5 hours!
Police brutality at Tehran protests - image from RevolutionaryRoad http://ping

KeyboardInterrupt: 

In [25]:
df = load_data()
print(df)

                   timestamp           user  \
0        2009-06-08 21:49:38          irzhy   
1        2009-06-08 22:00:30       andrejbr   
2        2009-06-11 16:56:50       deldel22   
3        2009-06-11 16:56:59     cameron987   
4        2009-06-11 16:57:53      testfiets   
...                      ...            ...   
2573072  2009-12-09 21:02:34   cherylfenton   
2573073  2009-12-09 21:02:47  zolanelsonbkz   
2573074  2009-12-09 21:02:50   dougjohnston   
2573075  2009-12-09 21:03:20      bryeaston   
2573076  2009-12-09 21:03:23  svenhohnsbeen   

                                                     tweet  
0        Каждый год в августе я меняю сотовый, похоже с...  
1        O que vocês acham: iPhone ou HTC TYTN II? #pes...  
2        How to cheat on any test easy http://bit.ly/10...  
3        @beatrizchavez A dork? NO way... you are. I ju...  
4        #iPhone: iPhone 3G S: de jacht op simlockvrije...  
...                                                    ...  
2573072 

In [62]:
import itertools
def extract_common_hash(df):
    pairs=[]
    def extract_hash_tags(tweet):
        return set([re.sub(r"(\W+)$", "", j) for j in set([i for i in tweet.replace(","," ").split() if i.startswith("#")])])
    for row in df.itertuples():
        tweet = row.tweet
        hashtags = extract_hash_tags(tweet)
        hashtags = set([i.lower() for i in hashtags])
        if len(list(hashtags))>1:
            #print(hashtags)
            for i in itertools.combinations(list(hashtags),2):
                if len(i[0])>0 and len(i[1])>0:
                    pairs.append(i)
    pairs=list(set(pairs))
    print(pairs[:10])
    return pairs
p=extract_common_hash(df)

[('#video', '#fume'), ('#nyc', '#yegtransit'), ('#arte', '#simpsons'), ('#usedbudget', '#gebührenfalle'), ('#thenines', '#fb'), ('#hiphop', '#rapper'), ('#wicca', '#pagan'), ('#windowsmobile', '#twitter'), ('#innovation', '#customer'), ('#3gs', '#purplesn0w')]


In [63]:
print(p[:100])
print(len(p))

[('#video', '#fume'), ('#nyc', '#yegtransit'), ('#arte', '#simpsons'), ('#usedbudget', '#gebührenfalle'), ('#thenines', '#fb'), ('#hiphop', '#rapper'), ('#wicca', '#pagan'), ('#windowsmobile', '#twitter'), ('#innovation', '#customer'), ('#3gs', '#purplesn0w'), ('#top', '#iphone'), ('#facebook', '#iphones'), ('#canon', '#flickr'), ('#ui', '#hero'), ('#phoenix', '#fail'), ('#viral', '#macbook'), ('#disorder', '#chaos'), ('#schumi', '#microsoft'), ('#enhanceexisting', '#developerwill'), ('#trends', '#gm'), ('#macbookpro', '#itunes'), ('#virigin', '#iphone'), ('#verizoniphone', '#dd'), ('#georgiatourism', '#eatonton'), ('#hilfreich', '#iphone'), ('#lies', '#truth'), ('#informática', '#notebook'), ('#smartphone', '#at&t'), ('#mmot', '#iphone'), ('#glee', '#a-choo'), ('#camerabag', '#horse'), ('#crmug', '#windowsmobile'), ('#27', '#iphone'), ('#slang', '#ttyl'), ('#fest8', '#phish'), ('#-apps', '#top'), ('#drugs', '#att'), ('#internet', '#browsing'), ('#catvsdog', '#nokiavsiphone'), ('#conce

In [84]:
f=open(f'hashtag_recs.txt', 'w+')
f.close()
for j in p[:50000]:
    with open(f'hashtag_recs.txt', 'a+') as file:
        try:
            file.write(f'{j[0]},{j[1]}')
            file.write("\n")
            file.close()
        except:
            continue

In [98]:
import networkx as nx
graph_path="hashtag_recs.txt"
df = pd.read_csv(graph_path, delimiter=",",names=["target","source"])
#df['source'] = df['source'].apply(lambda x: hash(x))
#df['target'] = df['target'].apply(lambda x: hash(x))
#df["weight"]=[1 for i in range(len(df))]
#df["timestamp"]=[i for i in range(len(df))]
df=df[:5000]
graph=nx.from_pandas_edgelist(df)
print(nx.number_of_nodes(graph))
#df.to_csv("data\hashtag_recs_larger.txt",sep=" ",index=False,header=False)
nx.write_gexf(graph,"hashtag_recs_tiny.gexf")

4905
